# 11 Iterator vs Iterable

We have seen the data structure such as List, Tuple, set, dictionary, etc, we said **they are all iterable**, so we can use for statement to loop these data structure. We have also seen generator, we said it's an iterator.

So what is the difference between iterator and iterable?

## 11.1 Iterators overview

**An iterator is an object that implements the iterator protocol**. In other words, an iterator is an object that implements the following methods:

- __iter__ returns the iterator object itself.
- __next__ returns the next element.

**Once you complete iterating a collection using an iterator, the iterator becomes exhausted. It means that you cannot use the iterator object again.**

## 11.2 Iterables overview

**An iterable is an object that you can iterate over. An object is iterable when it implements the __iter__ method.
And its __iter__ method returns a new iterator.**

## 11.3 Check built in python list

We have mentioned list is an iterable object. Let's check it first.

In [1]:
from typing import Iterable

# the numbers has type list, and is an instance of Iterable
numbers = [1, 2, 3]
print(type(numbers))
print(f"Is list an iterable {isinstance(numbers, Iterable)}")

# we use __iter__ to get the numbers iterator
number_iterator = numbers.__iter__()
# the type of returned iterator is list_iterator
print(type(number_iterator))

<class 'list'>
Is list an iterable True
<class 'list_iterator'>


In [2]:
next(number_iterator)

1

Because the list_iterator implements the __iter__ method, you can use the **built-in function iter()** to get the iterator object:
Note it's recommended to use function to get the iterator

Since the list_iterator also implements the __next__ method, you can use the built-in function next to iterate over the list:

In [3]:
n_iterator = iter(numbers)
next(n_iterator)

1

If we call many times of next(), we may receive a StopIteration exception. This is because the list iterator has been exhausted. To iterate the list again, you need to create a new iterator.

In [4]:
next(n_iterator)
next(n_iterator)
next(n_iterator)

StopIteration: 

The above example illustrates the reason why **the list is an iterable not an iterator**. The list is created once while the iterator is created every time you need to iterate over the list.

## 11.4 Simple implementation of an iterable and iterator

The below class Colors plays two roles: **iterable and iterator**.

The Colors class **is an iterator because it implements both __iter__ and __next__ method**. The __iter__ method returns the object itself. And the __next__ method returns the next item from a list.

The Colors class is **also an iterable because it implements the __iter__ method** that returns an object itself, which is an iterator.



In [5]:
class Colors:
    def __init__(self):
        self.colors = ['red', 'green', 'blue']
        self.__index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.__index >= len(self.colors):
            raise StopIteration
        color = self.colors[self.__index]
        self.__index += 1
        return color



In [6]:
colors = Colors()

for color in colors:
    print(color)

red
green
blue


Note, as colors is an iterator, after we iterate it once, the colors object becomes useless. If you attempt to iterate it again, you’ll get a StopIteration exception:

In [7]:
next(colors)

StopIteration: 

If you want to iterate again, you need to create a new Instance of Colors. This is quite annoying. How can we overcome this problem? Let's learn from the list implementation.

## 11.5 Separating an iterator from an iterable

Let’s separate the color iterator from its iterable like what Python does with the list iterator and list.

The following defines the Colors class:

In [8]:
class MyColors:
    def __init__(self):
        self.data = ['red', 'green', 'blue']

    def __len__(self):
        return len(self.data)

In [12]:
class ColorsIterator:
    def __init__(self, cols):
        self.__colors = cols
        self.__index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.__index >= len(self.__colors):
            raise StopIteration
        # here we access directly the data of class MyColors
        color = self.__colors.data[self.__index]
        self.__index += 1
        return color

How it works.

- The __init__ method accepts an iterable which is an instance of the Colors class.
- The __iter__ method returns the iterator itself.
- The __next__ method returns the next element from the Colors object.

The following shows how to use the ColorIterator to iterate over the Colors object:

In [13]:
colors = MyColors()
color_iterator = ColorsIterator(colors)

for color in color_iterator:
    print(color)

red
green
blue


There’s one problem!

When you want to iterate the MyColors object, you need to manually create a new ColorIterator object. And you also need to remember the iterator name ColorIterator.

It would be great if you can automate this. To do it, you can make the MyColors class iterable by implementing the __iter__ method:

In [14]:
class MyColors:
    def __init__(self):
        self.data = ['red', 'green', 'blue']

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        return ColorsIterator(self)

The __iter__ method returns a new instance of the ColorIterator class.

Now, you can iterate the Colors object without explicitly creating the ColorIterator object:

In [15]:
colors = MyColors()
for color in colors:
    print(color)

red
green
blue


Internally, the for loop calls the __iter__ method of the colors object to get the iterator and uses this iterator to iterate over the elements of the colors object.

As the ColorIterator can be only used on MyColors class. It's nature to put it inside the MyColors class. The following example places the ColorIterator class inside the Colors class to encapsulate them into a single class:

In [18]:
class MyColors:
    def __init__(self):
        self.data = ['red', 'green', 'blue']

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        return self.EmbedColorsIterator(self)

    # a private class
    class EmbedColorsIterator:
        def __init__(self, colors):
            self.__colors = colors
            self.__index = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self.__index >= len(self.__colors):
                raise StopIteration
            color = self.__colors.data[self.__index]
            self.__index += 1
            return color



In [19]:
colors = MyColors()
for color in colors:
    print(color)

red
green
blue


## 11.6 Summery

- An iterable is an object that implements the __iter__ method which returns an iterator.
- An iterator is an object that implements the __iter__ method which returns itself and the __next__ method which returns the next element.
- Iterators are also iterables. However, they’re iterables that become exhausted while iterables will never be exhausted.